# HW 2 - Starter Code

The database contains many tables / relations, but we will be mainly using 2 relations: Salaries and Teams. The schema (description of the fields in the tables) are given below. Not all columns/fields are relevant for this homework, and we will pointout relevant fields as we walk through the assignement. 

Salaries table
---------------
- yearID         Year
- teamID         Team
- lgID           League
- playerID       Player ID code
- salary         Salary


Teams Table
-----------
- yearID         Year
- lgID           League
- teamID         Team
- franchID       Franchise (links to TeamsFranchise table)
- divID          Team's division
- Rank           Position in final standings
- G              Games played
- GHome          Games played at home
- W              Wins
- L              Losses
- DivWin         Division Winner (Y or N)
- WCWin          Wild Card Winner (Y or N)
- LgWin          League Champion(Y or N)
- WSWin          World Series Winner (Y or N)
- R              Runs scored
- AB             At bats
- H              Hits by batters
- 2B             Doubles
- 3B             Triples
- HR             Homeruns by batters
- BB             Walks by batters
- SO             Strikeouts by batters
- SB             Stolen bases
- CS             Caught stealing
- HBP            Batters hit by pitch
- SF             Sacrifice flies
- RA             Opponents runs scored
- ER             Earned runs allowed
- ERA            Earned run average
- CG             Complete games
- SHO            Shutouts
- SV             Saves
- IPOuts         Outs Pitched (innings pitched x 3)
- HA             Hits allowed
- HRA            Homeruns allowed
- BBA            Walks allowed
- SOA            Strikeouts by pitchers
- E              Errors
- DP             Double Plays
- FP             Fielding  percentage
- name           Team's full name
- park           Name of team's home ballpark
- attendance     Home attendance total
- BPF            Three-year park factor for batters
- PPF            Three-year park factor for pitchers
- teamIDBR       Team ID used by Baseball Reference website
- teamIDlahman45 Team ID used in Lahman database version 4.5
- teamIDretro    Team ID used by Retrosheet

In [35]:
import sqlite3
import pandas

sqlite_file = 'lahman2014.sqlite'
conn = sqlite3.connect(sqlite_file)  # connect to database and ingest the tables 

# lets run a query to look at the data
salary_query = "SELECT yearID, sum(salary) as total_payroll FROM Salaries WHERE lgID == 'AL' GROUP BY yearID"

team_salaries = pandas.read_sql(salary_query, conn)
team_salaries.head()

,yearID,total_payroll
0,1985,134401120.0
1,1986,157716444.0
2,1987,136088747.0
3,1988,157049812.0
4,1989,188771688.0


## Analysis

We want to understand how efficient teams have been historically at spending money and getting wins in return. In the case of Moneyball, one would expect that Oakland was not much more efficient than other teams in their spending before 2000, were much more efficient (they made a movie about it after all) between 2000 and 2005, and by then other teams may have caught up. Lets see how this is reflected in the data we have.

### Relation Creation

Using SQL compute a new relation that contains a subset of fields of interest to help us compute further statisitcs of interest.  We neeed to think about the type of join used as it determines how missing data is handled. In the code below, a SQL statment is executed to create the new relation jusing a join between the two tables; 




In [36]:
createTable_query = "CREATE TABLE statsTbl AS SELECT \
Salaries.yearID, Teams.teamID, Teams.name, Salaries.salary, Teams.G, Teams.W, Teams.L \
FROM Salaries  \
JOIN Teams \
ON Salaries.yearID=Teams.yearID AND Salaries.teamID=Teams.teamID \
WHERE Salaries.lgID=='AL' ;"

cursor = conn.cursor()
cursor.execute(createTable_query)
conn.commit()

# if you run this mulitple times, you may see an error stating "statsTbl already exists". This error message can be ignored. 

OperationalError: table statsTbl already exists

In [37]:
query = "select * from statsTbl;"

result = pandas.read_sql(query, conn)
result.head()


# As you can see, it generates a new table/relation which contains 7 fields:
# yearID
# teamID
# name
# salary
# G --> denotes the number of games
# W --> denotes the number of wins
# L --> denotes the number of losses

,yearID,teamID,name,salary,G,W,L
0,1985,BAL,Baltimore Orioles,625000.0,161,83,78
1,1985,BAL,Baltimore Orioles,480000.0,161,83,78
2,1985,BAL,Baltimore Orioles,437500.0,161,83,78
3,1985,BAL,Baltimore Orioles,512500.0,161,83,78
4,1985,BAL,Baltimore Orioles,375000.0,161,83,78


Suppose we want to print the rows / entries for Oakland between 1988 - 1989. Hint: need to use a WHERE clause in the SQL query to filter out rows only for teamID="OAK"

In [38]:
query = "SELECT * FROM statsTbl WHERE teamID='OAK' AND yearID>=1988 AND yearID<=1989;"

result = pandas.read_sql(query, conn)
result.head()

,yearID,teamID,name,salary,G,W,L
0,1988,OAK,Oakland Athletics,495000.0,162,104,58
1,1988,OAK,Oakland Athletics,325000.0,162,104,58
2,1988,OAK,Oakland Athletics,550000.0,162,104,58
3,1988,OAK,Oakland Athletics,650000.0,162,104,58
4,1988,OAK,Oakland Athletics,100000.0,162,104,58


Suppose we want to print the year for which Oakland had the largest number of wins. Lets write that SQL query. 

In [39]:
query = "SELECT yearID, max(W) from statsTbl WHERE teamID='OAK';"

result = pandas.read_sql(query, conn)
result.head()

,yearID,max(W)
0,1988,104


Suppose we want to compute the total payroll for teamID='OAK' for the yearID='1988'. To do this, we must sum-up all the salaries for a given team for a given year. 

In [40]:
query = "SELECT teamID, yearID, SUM(salary) AS TotalSalary  \
FROM statsTbl  \
WHERE yearID='1988' AND teamID='OAK' \
GROUP BY teamID,yearID;"

result = pandas.read_sql(query, conn)
result.head()


,teamID,yearID,TotalSalary
0,OAK,1988,9690000.0


## Problem 1 

Using SQL, compute the result containing the total payroll and winning percentage (number of wins / number of games * 100) for each team (that is, for each teamID and yearID combination). 

Hint: Be sure to perform a groupby on the fields teamID and yearID



In [41]:
query = "SELECT yearID, teamID, SUM(salary) as TotalSalary, ((1.0*W/G)*100) as WinningPerc FROM statsTbl GROUP BY yearID, teamID;"

team_list = pandas.read_sql(query, conn)
team_list.head()

,yearID,teamID,TotalSalary,WinningPerc
0,1985,BAL,11560712.0,51.552795
1,1985,BOS,10897560.0,49.693252
2,1985,CAL,14427894.0,55.555556
3,1985,CHA,9846178.0,52.147239
4,1985,CLE,6551666.0,37.037037


## Problem 2

Write code to printout the teamID that had the highest WinningPercentage over total sepending (salaries). 

In [42]:
query = "SELECT teamID,  max((1.0*W/G)*100) as MaxWinningPerc FROM statsTbl;"

team_list = pandas.read_sql(query, conn)
team_list.head()

,teamID,MaxWinningPerc
0,SEA,71.604938


In [43]:
#SEA is the teamID that had the highest winning percentage.

In [44]:
query = "SELECT teamID,  max(((1.0*W/G)*100)/SUM(salary)) as MaxWinningPercOverSalaries FROM statsTbl;"

team_list = pandas.read_sql(query, conn)
team_list.head()

DatabaseError: Execution failed on sql 'SELECT teamID,  max(((1.0*W/G)*100)/SUM(salary)) as MaxWinningPercOverSalaries FROM statsTbl;': misuse of aggregate function SUM()

In [47]:
#Professor, I checked the parentheses multiple times, but I'm not sure why this isn't showing an error message.

## Problem 3

Write code to produce plots that illustrate the team's total spending (salaries) conditioned on time (from 1990-2014), specifically for teamID='OAK'.

In [45]:
query = "SELECT yearID, SUM(salary) as TotalSalary FROM statsTbl WHERE teamID='OAK' AND yearID>=1990 AND yearID<=2014 GROUP BY yearID;"

result = pandas.read_sql(query, conn)
result.head()



,yearID,TotalSalary
0,1990,19887501.0
1,1991,36999167.0
2,1992,41035000.0
3,1993,37812333.0
4,1994,34172500.0


In [49]:
createTable_query = "CREATE TABLE statsTbl AS SELECT \
Salaries.yearID, Teams.teamID, Teams.name, Salaries.salary, Teams.G, Teams.W, Teams.L \
FROM Salaries  \
JOIN Teams \
ON Salaries.yearID=Teams.yearID AND Salaries.teamID=Teams.teamID \
WHERE Salaries.lgID=='AL' ;"

cursor = conn.cursor()
cursor.execute(createTable_query)
conn.commit()


OperationalError: table statsTbl already exists

In [48]:
statsTbl.figure(tight_layout=True, figsize=(6, 4))
statsTbl.subplot(121)
statsTbl.scatter(statsTbl.YearID, statstbl.TotalSalary, color='k')
statsTbl.ylim(0, statsTbl.TotalSalary.max())
statsTbl.xlim(0, 5)
statsTbl.xlabel("YearID")
statsTbl.ylabel("TotalSalary")
remove_border()

statsTbl.subplot(122)
statsTbl.scatter(statsTbl.YearID, statsTbl.TotalSalary, color='k', alpha=.01)
statsTbl.ylim(0, statsTbl.TotalSalary.max())
statsTbl.xlim(0, 5)

statsTbl.xlabel("YearID")
statsTbl.ylabel("TotalSalary")
remove_border()


NameError: name 'statsTbl' is not defined